In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWhrzpVTc_WubTS3Ai0zgPTJYr25yp3WfOJ5zWc2Z51uAY3jTJFk4OY
Mounted at /content/drive


In [2]:
pip install neattext

     |████████████████████████████████| 112kB 27.0MB/s 


In [3]:
import pandas as pd

import tkinter
from tkinter.filedialog import askopenfilename
import string #Opérations courantes sur les chaînes
import re   # pour utiliser  des expressions régulières ex:$
import nltk #boites outils ntk
from sklearn.model_selection import train_test_split
import neattext as nt
import neattext.functions as nfx
from neattext import TextCleaner as tc

In [4]:
#Cleaning & Preprocessing
def process(text):
# la normalisation
    regex = re.compile(r'[إأٱآا]') #sauvegarder l'expression réguliere (r=expression reguliere)
    text = re.sub(regex, 'ا', str(text)) #remblacer des expressions par d'autres dans varia texte
    regex = re.compile(r'[ى]')
    text = re.sub(regex, 'ي', str(text))
    regex = re.compile(r'[ؤئ]')
    text = re.sub(regex, 'ء', str(text))
    text = re.sub('\n', ' ', str(text))
    text = re.sub(' و ', ' ', str(text))
    text = re.sub(r'\s*[A-Za-z]+\b', '' , text)
    text = text.rstrip()
#supprimer les diacritiques
    arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)   #verbose: séparer visuellement les expre et ajouter des commentaires
    text = re.sub(arabic_diacritics, '', text)
    

#supprimer les ponctuations
    #liste des ponctuations arabe et anglais qu'on veut supprimer
    arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''    # utiliser ''' :multiples caractére de chaine
    english_punctuations = string.punctuation              #(!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~)
    punctuations_list = arabic_punctuations + english_punctuations
    #table de mappage
    translator = str.maketrans('','', punctuations_list) # le 1er elem a remplacer/le 2eme le remplacement,3eme=supprimer
    text= text.translate(translator) #chaine ou les caractere sont remplacé par d'autres dns table de mappage

#supprimer les chaines répétés
    text=re.sub(r'(.)\1+', r'\1', str(text)) 

#supprimer les mots vides
    stopwords=nltk.corpus.stopwords.words('arabic') #les mots vides en arabe
    txt=[word for word in text if word not in stopwords]

#supprimer les émojis
    myre = re.compile(u'['
    u'\U0001F300-\U0001F64F'
    u'\U0001F680-\U0001F6FF'     
    u'\u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
    text=myre.sub('', text)
    
    
    return text

In [5]:
filename='new_db.xlsx'

In [6]:
#Importer le fichier excel
data=pd.read_excel(filename) #filename:le chemin d'acces au fichier excel

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Cleaning DB
data['TWEET'] = data['TWEET'].apply(process)
data['TWEET']=data['TWEET'].apply(nfx.remove_hashtags)
data['TWEET']=data['TWEET'].apply(nfx.remove_userhandles)

In [9]:
index=[]
for i in range(len(data)):
  if data.iloc[i].TWEET=='':
    index.append(i)

for i in index:
  data=data.drop(i)
data = data.reset_index().drop('index',axis=1)

In [10]:
#Affichage des premiers elts
data.head()

,TWEET,LABEL
0,الاوليمبياد الجايه هكون لسه ف الكليه,0
1,عجز الموازنه وصل ل937 من الناتج المحلي يعني لس...,1
2,كتنا نيله ف حظنا الهباب,2
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,3
4,الاوليمبياد نظامها مختلف ومواعيد المونديال مكا...,0


In [11]:
#Affichage des derniers elts
data.tail()

,TWEET,LABEL
29947,اتمني تعملو بطاقات خاصة او ذهبية لعملاء الداءم...,3
29948,المشويات عندهم اطيب مشويات اكلتها بحياتي والاس...,3
29949,المطعم متاز لغايه من ناحيه تجهيزات المطعم وايض...,3
29950,جينا عندهم الساعة 10 باليل مافي احد اتبرع يشوف...,1
29951,اذا كان الي بالشارع التجاري اوه ماشاء اله واله...,3


In [12]:
#Afficher les labels de la bd
data.LABEL.unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [13]:
#Remplacer chaque label par un entier
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x]=len(encode_dict)
    return encode_dict[x]

data['LABEL'] = data['LABEL'].apply(lambda x: encode_cat(x))

In [14]:
#Split data into random train(70%) and test(30%) subsets
text_train, text_test, label_train, label_test = train_test_split(data.TWEET,data.LABEL,test_size=0.20, random_state=42)
print(text_train.shape,text_test.shape,label_train.shape,label_test.shape)  

(23961,) (5991,) (23961,) (5991,)


# GLOVE

# Preparation du model

In [15]:
#from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

In [16]:
word2vec_output_file = 'vectors.txt.word2vec'

In [17]:
# load the Stanford GloVe model
%cd /content/drive/MyDrive/GloVe
gl = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
%cd ..
%cd ..
%cd ..

/content/drive/MyDrive/GloVe
/content/drive/My Drive
/content/drive
/content


# Machine learning



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [ ]:
# We will create a class that will contain our vocabulary and glove vectors and then it will transform every review 
#(sentence) to a vector representation as we describe previously.
class Word2VecVectorizer:
  def __init__(self, gl):
    self.word_vectors = gl

  def fit(self, data):
    pass

  def transform(self, data):
    X = np.zeros((len(data), gl.vector_size))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [ ]:
# Set a word vectorizer
vectorizer = Word2VecVectorizer(gl)
# Get the sentence embeddings for the train dataset
text_train_glove = vectorizer.fit_transform(text_train)
# Get the sentence embeddings for the test dataset
text_test_glove = vectorizer.transform(text_test)

In [ ]:
# create the model, train it, print scores
clf = LogisticRegression(solver='liblinear')

clf.fit(text_train_glove, label_train)

tag_pred = clf.predict(text_test_glove)

print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.60      0.63      0.61       314
           1       0.56      0.56      0.56      1184
           2       0.54      0.33      0.41       701
           3       0.67      0.87      0.76      2434
           4       0.50      0.17      0.25       470
           5       0.70      0.49      0.58       381
           6       0.77      0.71      0.74       231
           7       0.85      0.71      0.77       276

    accuracy                           0.64      5991
   macro avg       0.65      0.56      0.59      5991
weighted avg       0.63      0.64      0.62      5991



In [ ]:
# create the model, train it, print scores
SVM=svm.SVC()

SVM.fit(text_train_glove, label_train)

tag_pred = SVM.predict(text_test_glove)

print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.59      0.84      0.69       314
           1       0.59      0.66      0.63      1184
           2       0.57      0.39      0.46       701
           3       0.72      0.86      0.78      2434
           4       0.63      0.20      0.31       470
           5       0.75      0.49      0.59       381
           6       0.81      0.71      0.76       231
           7       0.91      0.71      0.79       276

    accuracy                           0.68      5991
   macro avg       0.70      0.61      0.63      5991
weighted avg       0.68      0.68      0.66      5991



In [ ]:
# create the model, train it, print scores
rf=RandomForestClassifier()

rf.fit(text_train_glove, label_train)

tag_pred = rf.predict(text_test_glove)

print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.58      0.69      0.63       314
           1       0.78      0.73      0.75      1184
           2       0.79      0.54      0.64       701
           3       0.73      0.93      0.82      2434
           4       0.78      0.48      0.59       470
           5       0.73      0.57      0.64       381
           6       0.85      0.56      0.68       231
           7       0.81      0.57      0.67       276

    accuracy                           0.74      5991
   macro avg       0.76      0.63      0.68      5991
weighted avg       0.75      0.74      0.73      5991



In [ ]:
nb=GaussianNB()

nb.fit(text_train_glove, label_train)

tag_pred = nb.predict(text_test_glove)

print(classification_report(label_test, tag_pred))

              precision    recall  f1-score   support

           0       0.50      0.69      0.58       314
           1       0.31      0.77      0.44      1184
           2       0.28      0.07      0.11       701
           3       0.79      0.25      0.38      2434
           4       0.19      0.18      0.18       470
           5       0.44      0.46      0.45       381
           6       0.44      0.66      0.53       231
           7       0.35      0.63      0.45       276

    accuracy                           0.40      5991
   macro avg       0.41      0.46      0.39      5991
weighted avg       0.52      0.40      0.37      5991



# Deep Learning

In [18]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.metrics import classification_report

In [19]:
#create a word-to-index dictionary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_test = tokenizer.texts_to_sequences(text_test)

In [20]:
# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

maxlen = 82

#Add 0 at the end of the list until it reaches the max length
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [21]:
#Create an embedding matrix where each row number will correspond to the index of the word in the corpus
embedding_matrix = np.zeros((vocab_size, 256))
for word, index in tokenizer.word_index.items():
  try:
    embedding_vector = gl.get_vector(word)
  except:
    continue
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

In [ ]:
#--------------------------------------------------------------------------------------------------------------

In [22]:
model1 = Sequential()
embedding_layer = Embedding(vocab_size,256 , weights=[embedding_matrix], input_length=maxlen , trainable=False)
model1.add(embedding_layer)

# The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
model1.add(GRU(256, activation='sigmoid',recurrent_activation='sigmoid' , return_sequences=True))

# The output of SimpleRNN will be a 2D tensor of shape (batch_size, 128)
model1.add(SimpleRNN(128))


model1.add(Dense(8, activation='sigmoid'))
model1.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 82, 256)           16857344  
_________________________________________________________________
gru (GRU)                    (None, 82, 256)           394752    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
Total params: 17,302,408
Trainable params: 445,064
Non-trainable params: 16,857,344
_________________________________________________________________


In [23]:
history = model1.fit(X_train, label_train, batch_size=128, epochs=30, verbose=1, validation_split=0.2)

score = model1.evaluate(X_test, label_test, verbose=1)

Epoch 1/30
150/150 [==============================] - 61s 254ms/step - loss: 1.7784 - acc: 0.4006 - val_loss: 1.7148 - val_acc: 0.4271
Epoch 2/30
150/150 [==============================] - 35s 236ms/step - loss: 1.6985 - acc: 0.4323 - val_loss: 1.6372 - val_acc: 0.4381
Epoch 3/30
150/150 [==============================] - 36s 240ms/step - loss: 1.6044 - acc: 0.4372 - val_loss: 1.4829 - val_acc: 0.4646
Epoch 4/30
150/150 [==============================] - 35s 233ms/step - loss: 1.4663 - acc: 0.4684 - val_loss: 1.3508 - val_acc: 0.5416
Epoch 5/30
150/150 [==============================] - 37s 246ms/step - loss: 1.3321 - acc: 0.5343 - val_loss: 1.2491 - val_acc: 0.5642
Epoch 6/30
150/150 [==============================] - 37s 249ms/step - loss: 1.2069 - acc: 0.5722 - val_loss: 1.1410 - val_acc: 0.5961
Epoch 7/30
150/150 [==============================] - 36s 239ms/step - loss: 1.1370 - acc: 0.5981 - val_loss: 1.1005 - val_acc: 0.6149
Epoch 8/30
150/150 [==============================] - 3

In [24]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.813711941242218
Test Accuracy: 0.7269237041473389


In [ ]:
pred = np.argmax(model1.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.56      0.86      0.68       314
           1       0.64      0.68      0.66      1184
           2       0.49      0.50      0.50       701
           3       0.84      0.85      0.85      2434
           4       0.40      0.26      0.32       470
           5       0.64      0.48      0.55       381
           6       0.73      0.63      0.68       231
           7       0.83      0.82      0.83       276

    accuracy                           0.70      5991
   macro avg       0.64      0.64      0.63      5991
weighted avg       0.69      0.70      0.69      5991



In [ ]:
#----------------------------------------------------------------------------------

In [ ]:
model2 = Sequential()
embedding_layer = Embedding(vocab_size, 256, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model2.add(embedding_layer)

model2.add(LSTM(1024))

model2.add(Dense(8, activation='softmax'))
model2.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model2.fit(X_train, label_train, batch_size=128, epochs=100, verbose=1, validation_split=0.2)

score = model2.evaluate(X_test, label_test, verbose=1)

Epoch 1/100
150/150 [==============================] - 22s 122ms/step - loss: 1.7990 - acc: 0.4067 - val_loss: 1.4860 - val_acc: 0.4367
Epoch 2/100
150/150 [==============================] - 17s 117ms/step - loss: 1.4536 - acc: 0.4674 - val_loss: 1.3816 - val_acc: 0.5122
Epoch 3/100
150/150 [==============================] - 17s 111ms/step - loss: 1.3686 - acc: 0.5102 - val_loss: 1.3741 - val_acc: 0.4884
Epoch 4/100
150/150 [==============================] - 17s 111ms/step - loss: 1.2928 - acc: 0.5416 - val_loss: 1.2978 - val_acc: 0.5602
Epoch 5/100
150/150 [==============================] - 17s 111ms/step - loss: 1.2248 - acc: 0.5689 - val_loss: 1.2011 - val_acc: 0.5756
Epoch 6/100
150/150 [==============================] - 17s 112ms/step - loss: 1.1616 - acc: 0.6002 - val_loss: 1.1501 - val_acc: 0.5900
Epoch 7/100
150/150 [==============================] - 17s 113ms/step - loss: 1.1302 - acc: 0.6006 - val_loss: 1.1652 - val_acc: 0.5992
Epoch 8/100
150/150 [===========================

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 1.5770083665847778
Test Accuracy: 0.7614755630493164


In [ ]:
pred = np.argmax(model2.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.62      0.55      0.59       314
           1       0.75      0.74      0.74      1184
           2       0.65      0.64      0.65       701
           3       0.84      0.89      0.86      2434
           4       0.56      0.56      0.56       470
           5       0.70      0.66      0.68       381
           6       0.81      0.73      0.77       231
           7       0.89      0.79      0.84       276

    accuracy                           0.76      5991
   macro avg       0.73      0.70      0.71      5991
weighted avg       0.76      0.76      0.76      5991



In [ ]:
#------------------------------------------------------

In [ ]:
model3 = Sequential()

embedding_layer = Embedding(vocab_size, 256, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model3.add(embedding_layer)


model3.add(Conv1D(512, 1, activation='softsign'))
model3.add(Conv1D(512, 1, activation='softsign'))
model3.add(GlobalMaxPooling1D())
model3.add(Dense(8, activation='softplus'))
model3.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
history = model3.fit(X_train, label_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

score = model3.evaluate(X_test, label_test, verbose=1)

Epoch 1/10
150/150 [==============================] - 4s 21ms/step - loss: 1.6150 - acc: 0.3821 - val_loss: 1.2126 - val_acc: 0.4555
Epoch 2/10
150/150 [==============================] - 3s 18ms/step - loss: 1.1528 - acc: 0.4696 - val_loss: 1.0839 - val_acc: 0.5074
Epoch 3/10
150/150 [==============================] - 3s 18ms/step - loss: 0.9995 - acc: 0.5638 - val_loss: 0.8561 - val_acc: 0.7085
Epoch 4/10
150/150 [==============================] - 3s 18ms/step - loss: 0.7604 - acc: 0.7358 - val_loss: 0.7946 - val_acc: 0.7242
Epoch 5/10
150/150 [==============================] - 3s 19ms/step - loss: 0.6829 - acc: 0.7628 - val_loss: 0.7559 - val_acc: 0.7363
Epoch 6/10
150/150 [==============================] - 3s 18ms/step - loss: 0.6192 - acc: 0.7907 - val_loss: 0.7318 - val_acc: 0.7492
Epoch 7/10
150/150 [==============================] - 3s 18ms/step - loss: 0.5613 - acc: 0.8116 - val_loss: 0.7226 - val_acc: 0.7501
Epoch 8/10
150/150 [==============================] - 3s 18ms/step - 

In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

Test Score: 0.6887174248695374
Test Accuracy: 0.7698214054107666


In [ ]:
pred = np.argmax(model3.predict(X_test), axis=-1)
print(classification_report(label_test, pred))

              precision    recall  f1-score   support

           0       0.62      0.74      0.68       314
           1       0.71      0.78      0.75      1184
           2       0.67      0.63      0.65       701
           3       0.84      0.88      0.86      2434
           4       0.64      0.50      0.56       470
           5       0.75      0.60      0.66       381
           6       0.85      0.76      0.80       231
           7       0.96      0.87      0.91       276

    accuracy                           0.77      5991
   macro avg       0.76      0.72      0.73      5991
weighted avg       0.77      0.77      0.77      5991



In [ ]:
#--------------------------------------------------------------------------------------------